# MaFaulDa Experiments

This notebook contains the basic code to reconstruct the results shown in the paper, in terms of feature selection, model selection and model evaluation.

In [ ]:
import os

import torch
import pandas as pd
import numpy as np

from kan import KAN
from kan.utils import ex_round

from utils import get_data, feature_selection, model_selection, plot_heatmaps, plot_pareto, plot_cm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

The setup is based on the classification problem, but it can be trivially extended to fault detection or severity classification by wrangling the `df` dataframe. An example for fault detection is provided in the commented code.

In [ ]:
experiment_name = 'mafaulda_classification'
exp_seed = 42 # Add here the random seed

# Read the data
df = pd.read_csv(os.path.join('data','mafaulda.csv')).drop(columns=['severity'])

# The following is an example of trivial wrangling to study the fault detection problem instead of classification
"""
# Keep all rows with label 0
zeros_df = df[df['label'] == 0]

# Filter out label 0 and sample 60 rows for each label from 1 to 9
filtered_df = df[df['label'] != 0]
sampled_df = filtered_df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 60), random_state=exp_seed))

# Concatenate the two dataframes (zeros_df and sampled_df)
df = pd.concat([zeros_df, sampled_df]).reset_index(drop=True)

# Cast labels to binary format
df['label'] = df['label'].apply(lambda x: 1 if x != 0 else 0)
"""

## Feature Selection

We run a grid search over $\lambda$ and $\tau$ to find the combination that retains the smallest number of features, with the highest attained F1-Score.

In [ ]:
# Parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

grid_size = 5
grid_eps = 0.05
k = 3
epochs = 80
use_scaler = True
data_split = (70,15,15)
optim = "Adam"

thresholds = np.linspace(0.01, 0.1, 20)
lambdas = np.linspace(0.001, 0.01, 20)

featsdf = feature_selection(df=df, grid_size=grid_size, grid_eps=grid_eps, k=k,
                            thresholds=thresholds, lambdas=lambdas, optim=optim, epochs=epochs, 
                            use_scaler=use_scaler, data_split=data_split, 
                            device=device, exp_seed=exp_seed)

In [ ]:
# Create the results directory if it doesn't exist
os.makedirs('results', exist_ok=True)

# Save the dataframe locally as pickle file to avoid having to run everything again
featsdf.to_pickle(os.path.join('results', f'{experiment_name}_featsdf.pkl'))

The results of the feature selection process in terms of the grid search, as well as the generated Pareto set can be seen in the following.

In [ ]:
indices = {'x': 'thresholds', 'y': 'lambdas', 'z0': 'f1_scores', 'z1': 'num_feats'}
savepath = os.path.join('results', f'{experiment_name}_feat_heatmaps.pdf')
titles = ['Interpolated F1-Score', 'Interpolated Number of Features']
x_label, y_label = r'$\tau$ Values', r'$\lambda$ Values'
cbar_labels = ['','']

plot_heatmaps(featsdf, indices, savepath, interpolation='spline36', cmap='Spectral', titles=titles, x_label=x_label, y_label=y_label, cbar_labels=cbar_labels)

In [ ]:
cmap = plt.get_cmap('Spectral')

plotcols = ['num_feats', 'f1_scores', 'pareto']
savepath = os.path.join('results', f'{experiment_name}_feat_pareto.pdf')
bg_col, pareto_col, nonpareto_col = '#f7fafa', cmap(0.25), cmap(1.0)
labels=['Points outside of the Pareto set', 'Points within the Pareto set']
title='F1-Score vs Number of Retained Features'
x_label, y_label = 'Number of Features', 'F1-Score (%)'

plot_pareto(featsdf, savepath, plotcols=plotcols, bg_col=bg_col, pareto_col=pareto_col, nonpareto_col=nonpareto_col,
            labels=labels, title=title, x_label=x_label, y_label=y_label)

Given the returned Pareto Set, we decide to work with the feature set that has the highest F1-Score, but contains at most 10 features.

In [ ]:
# Get the index corresponding to the "best" point of the pareto set
# Objective: Maximize F1-Score keeping up to 10 features

# Get pareto set
fpset = featsdf[featsdf['pareto']==True]

# Select points with num_feats <= 10
under_10_pset = fpset.loc[fpset['num_feats'] <= 10]

# Get the index of the highest F1-Score among these points
idx = under_10_pset.loc[under_10_pset['f1_scores'] == under_10_pset['f1_scores'].max()].index

In [ ]:
fpset

This means that feature selection is complete and we are working with the following features:

In [ ]:
kept_feats = featsdf.iloc[idx]['features'].values[0].tolist()
lamb = featsdf.iloc[idx]['lambdas'].values[0]
print(f"Working with the {len(kept_feats)} following features:\n{kept_feats} \nand lambda = {lamb:.3f}.")

## Model Selection

We run another grid search, this time for the model parameters $G$ and $g_e$, using only the retained features.

In [ ]:
if use_scaler == True:
    scaler = StandardScaler()
else:
    scaler = None

dataset = get_data(df, scaler=scaler, data_split=data_split, final_eval=False, feat_idxs=kept_feats, device=device, exp_seed=exp_seed)

k = 4
alpha_param = 0.05
beta_param = 1.5
r2_threshold = 0.0

optim = "Adam"
epochs = 200
grid_update_num = 10
stop_grid_update_step = 150

grid_sizes = [8, 10, 12, 15, 20, 30, 40, 50]
grid_es = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

modelsdf = model_selection(dataset, grid_sizes, grid_es, lamb=0.0, k=k, optim=optim, epochs=epochs, grid_update_num=grid_update_num,
                           stop_grid_update_step=stop_grid_update_step, alpha=alpha_param, beta=beta_param,
                           r2_threshold=r2_threshold, device=device, exp_seed=exp_seed)

In [ ]:
# Separate the grid search for adaptive grids and the one for non adaptive (g_e = 1.0)
staticmodelsdf = modelsdf[modelsdf['grid_es'] == 1.0].reset_index(drop=True)
modelsdf = modelsdf[modelsdf['grid_es'] != 1.0].reset_index(drop=True)

# Save the dataframe locally as pickle file to avoid having to run everything again
modelsdf.to_pickle(os.path.join('results', f'{experiment_name}_modelsdf.pkl'))

As previously, the results of the model selection process in terms of the grid search, as well as the generated Pareto set can be seen in the following.

In [ ]:
indices = {'x': 'grid_es', 'y': 'grid_sizes', 'z0': 'f1_kan', 'z1': 'f1_sym'}
savepath = os.path.join('results', f'{experiment_name}_model_heatmaps.pdf')
titles = ['Interpolated KAN F1-Score', 'Interpolated Symbolic F1-Score']
x_label, y_label = 'Grid Adaptability', 'Grid Size'
cbar_labels = ['','']

plot_heatmaps(modelsdf, indices, savepath, interpolation='spline36', cmap='Spectral', titles=titles, x_label=x_label, y_label=y_label, cbar_labels=cbar_labels)

In [ ]:
cmap = plt.get_cmap('Spectral')

plotcols = ['f1_kan', 'f1_sym', 'pareto']
savepath = os.path.join('results', f'{experiment_name}_model_pareto.pdf')
bg_col, pareto_col, nonpareto_col = '#f7fafa', cmap(0.25), cmap(1.0)
labels=['Points outside of the Pareto set', 'Points within the Pareto set']
title='Achieved F1-Scores per run'
x_label, y_label = 'KAN F1-Score (%)', 'Symbolic F1-Score (%)'


plot_pareto(modelsdf, savepath, plotcols=plotcols, bg_col=bg_col, pareto_col=pareto_col, nonpareto_col=nonpareto_col,
            labels=labels, title=title, x_label=x_label, y_label=y_label)

Given the returned Pareto Set, we decide to work with the model that has the highest Symbolic F1-Score.

In [ ]:
# Get the index corresponding to the "best" point of the pareto set
# Objective: Maximize Mean F1-Score

# Get pareto set
mpset = modelsdf[modelsdf['pareto']==True]

# Calculate mean F1-Score
mean_f1 = 0.5*(mpset['f1_kan'] + mpset['f1_sym'])

# Select point with maximum Symbolic F1-Score
idx = mean_f1[mean_f1 == mean_f1.max()].index

In [ ]:
mpset

This suggests that model selection is complete and we are working with the following hyperparameters:

In [ ]:
G, g_e = modelsdf.iloc[idx]['grid_sizes'].values[0], modelsdf.iloc[idx]['grid_es'].values[0]
print(f"Working with G = {G} and g_e = {g_e}.")

## Final Evaluation

To perform the final evaluation with the selected features and model hyperparameters, we concatenate the training and validation data and use them to train a final model instance. Then, we assess its performance on the test data, which have not been used anywhere so far.

In [ ]:
# Concatenate validation and training data
if use_scaler == True:
    final_scaler = StandardScaler()
else:
    final_scaler = None
    
final_data = get_data(df, scaler=final_scaler, data_split=data_split, final_eval=True, feat_idxs=kept_feats, device=device, exp_seed=exp_seed)

# Initialize a model instance
kan_input = final_data['train_input'].shape[1]
kan_output = final_data['train_label'].unique().shape[0]
model = KAN(width=[kan_input, kan_output], grid=G, k=4, grid_eps=g_e, sparse_init=False, seed=exp_seed, auto_save=False, device=device)

# Train Model
results = model.fit(final_data, opt=optim, steps=epochs, lamb=0.0, update_grid=True, grid_update_num=grid_update_num,
                    stop_grid_update_step=stop_grid_update_step, loss_fn=torch.nn.CrossEntropyLoss())

print(f"Trained final model with {len(kept_feats)} features, using G = {G} and g_e = {g_e}.")

In [ ]:
# Evaluate on test data
preds = torch.argmax(model.forward(final_data['test_input']).detach(), dim=1).cpu()
truth = final_data['test_label'].cpu()

# Get classification report
print(classification_report(truth, preds))

# Plot Confusion Matrix
class_names = ["N", "F",]
cm_title = 'Confusion Matrix for Fault Classification'
x_label = 'Predicted Class'
y_label = 'True Class'
plot_cm(preds, truth, class_names, percs=False, cmap='Blues', title=cm_title, x_label=x_label, y_label=y_label)

The final step is the "symbolification" of the trained KAN and its re-evaluation.

In [ ]:
# Symbolify Model
model.auto_symbolic(verbose=1, alpha=alpha_param, beta=beta_param, r2_threshold=r2_threshold)

# Evaluate Symbolic Version
preds_sym = torch.argmax(model.forward(final_data['test_input']).detach(), dim=1).cpu()

# Get classification report
print(classification_report(truth, preds_sym))

# Plot Confusion Matrix
plot_cm(preds_sym, truth, class_names, percs=False, cmap='Blues', title=cm_title, x_label=x_label, y_label=y_label)

Extracting the symbolic formulas:

In [ ]:
symbf = model.symbolic_formula()[0]

for i in range(len(symbf)):
    sf = symbf[i]
    print(ex_round(sf, 3))